In [145]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
import datetime
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.austintexas.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.austintexas.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("fdj4-gpfu", limit=200000,)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [146]:
results_df.head()

,incident_report_number,crime_type,ucr_code,family_violence,occ_date_time,occ_date,occ_time,rep_date_time,rep_date,rep_time,...,longitude,location,:@computed_region_a3it_2a2z,:@computed_region_8spj_utxs,:@computed_region_q9nd_rr82,:@computed_region_qwte_z96m,ucr_category,category_description,clearance_status,clearance_date
0,2022813090,ASSAULT BY CONTACT,902,N,2022-01-08T23:57:00.000,2022-01-08T00:00:00.000,2357,2022-01-08T23:57:00.000,2022-01-08T00:00:00.000,2357,...,-97.73818499,"{'latitude': '30.25953309', 'longitude': '-97....",2856,9,10,1405,NaN,NaN,NaN,NaN
1,202281353,FAMILY DISTURBANCE,3400,N,2022-01-08T23:51:00.000,2022-01-08T00:00:00.000,2351,2022-01-09T00:04:00.000,2022-01-09T00:00:00.000,4,...,-97.71813346,"{'latitude': '30.21951765', 'longitude': '-97....",3256,3,3,1599,NaN,NaN,NaN,NaN
2,202281199,AGG ROBBERY/DEADLY WEAPON,300,N,2022-01-08T23:37:00.000,2022-01-08T00:00:00.000,2337,2022-01-08T23:37:00.000,2022-01-08T00:00:00.000,2337,...,-97.71524279,"{'latitude': '30.42489395', 'longitude': '-97....",3243,7,7,3749,120,Robbery,NaN,NaN
3,202281282,FAMILY DISTURBANCE,3400,N,2022-01-08T23:34:00.000,2022-01-08T00:00:00.000,2334,2022-01-08T23:34:00.000,2022-01-08T00:00:00.000,2334,...,-97.72062928,"{'latitude': '30.22899126', 'longitude': '-97....",3256,3,3,102,NaN,NaN,NaN,NaN
4,202281324,AUTO THEFT,700,N,2022-01-08T23:31:00.000,2022-01-08T00:00:00.000,2331,2022-01-08T23:31:00.000,2022-01-08T00:00:00.000,2331,...,-97.71745933,"{'latitude': '30.35094052', 'longitude': '-97....",3645,7,7,NaN,240,Auto Theft,NaN,NaN


In [147]:
results_df.dtypes

incident_report_number         object
crime_type                     object
ucr_code                       object
family_violence                object
occ_date_time                  object
occ_date                       object
occ_time                       object
rep_date_time                  object
rep_date                       object
rep_time                       object
location_type                  object
address                        object
zip_code                       object
council_district               object
sector                         object
district                       object
pra                            object
census_tract                   object
x_coordinate                   object
y_coordinate                   object
latitude                       object
longitude                      object
location                       object
:@computed_region_a3it_2a2z    object
:@computed_region_8spj_utxs    object
:@computed_region_q9nd_rr82    object
:@computed_r

In [148]:
crime_df = results_df[['crime_type', 'family_violence', 'occ_date', 'occ_time', 'zip_code', 'latitude', 'longitude']]
crime_df.dropna(inplace=True)
crime_df

C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,crime_type,family_violence,occ_date,occ_time,zip_code,latitude,longitude
0,ASSAULT BY CONTACT,N,2022-01-08T00:00:00.000,2357,78701,30.25953309,-97.73818499
1,FAMILY DISTURBANCE,N,2022-01-08T00:00:00.000,2351,78741,30.21951765,-97.71813346
2,AGG ROBBERY/DEADLY WEAPON,N,2022-01-08T00:00:00.000,2337,78727,30.42489395,-97.71524279
3,FAMILY DISTURBANCE,N,2022-01-08T00:00:00.000,2334,78741,30.22899126,-97.72062928
4,AUTO THEFT,N,2022-01-08T00:00:00.000,2331,78757,30.35094052,-97.71745933
...,...,...,...,...,...,...,...
199995,POSS CONTROLLED SUB/NARCOTIC,N,2019-12-16T00:00:00.000,15,78741,30.21620096,-97.69975739
199996,FAMILY DISTURBANCE,N,2019-12-16T00:00:00.000,6,78758,30.37194381,-97.71780314
199997,ASSAULT W/INJURY-FAM/DATE VIOL,Y,2019-12-16T00:00:00.000,4,78704,30.24143168,-97.73797773
199998,CRIMINAL MISCHIEF,N,2019-12-16T00:00:00.000,1,78758,30.40485376,-97.71371186


In [149]:
#Converting time data to datetime

timefmt_date = "%Y-%m-%d"
timefmt_time = "%H:%M" 
crime_df['occ_date']= pd.to_datetime(crime_df['occ_date'], format = timefmt_date, errors='coerce')

time_occ = []

# Loop to format data for date time
for str in crime_df['occ_time']:
          if len(str) == 4 :
                    val =  str[:2] + ':' +str[2:]
          elif len(str) == 3:
                    val = str[:1] + ':' +str[1:]
          elif len(str) == 2:
                    val = '00:' + str
          elif len(str) == 1:
                    val = '00:0' + str
          time_occ.append(val)
#
crime_df['time_occ'] = time_occ

crime_df['time_occ'] = pd.to_datetime(crime_df['time_occ'], format = timefmt_time, errors='coerce')
crime_df['time_occ'] = [time.time() for time in crime_df['time_occ']]
crime_df.head()
          

C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

KeyboardInterrupt: 

In [ ]:
crime_df.groupby(['crime_type']).count()

,family_violence,occ_date,occ_time,zip_code,latitude,longitude,time_occ
crime_type,,,,,,,
ABUSE OF 911,62,62,62,62,62,62,62
ABUSE OF CORPSE,1,1,1,1,1,1,1
AGG ASLT ENHANC STRANGL/SUFFOC,102,102,102,102,102,102,102
AGG ASLT STRANGLE/SUFFOCATE,1210,1210,1210,1210,1210,1210,1210
AGG ASLT W/MOTOR VEH FAM/DAT V,46,46,46,46,46,46,46
...,...,...,...,...,...,...,...
VOCO - ALCOHOL CONSUMPTION,24,24,24,24,24,24,24
VOCO AMPLIFIED MUSIC/VEHICLE,1,1,1,1,1,1,1
VOYEURISM-PEEPING TOM,10,10,10,10,10,10,10


In [ ]:
crime_df = crime_df.loc[(crime_df['occ_date']>= '2020-01-01') & (crime_df['occ_date']< '2022-01-01')]
crime_df

,crime_type,family_violence,occ_date,occ_time,zip_code,latitude,longitude,time_occ
1115,PUBLIC INTOXICATION,N,2021-12-31,2350,78701,30.2672999,-97.73885705,23:50:00
1116,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,2350,78701,30.26373894,-97.74365146,23:50:00
1117,THEFT,N,2021-12-31,2350,78741,30.21526412,-97.70301942,23:50:00
1118,DOC DISCHARGE GUN - PUB PLACE,N,2021-12-31,2347,78752,30.32804875,-97.69368286,23:47:00
1119,AGG ASLT STRANGLE/SUFFOCATE,Y,2021-12-31,2340,78758,30.36926216,-97.69510501,23:40:00
...,...,...,...,...,...,...,...,...
195555,THEFT OF BICYCLE,N,2020-01-01,0,78703,30.27315833,-97.76014527,00:00:00
195556,HARASSMENT,N,2020-01-01,0,78741,30.2156655,-97.70354685,00:00:00
195557,THEFT FROM AUTO,N,2020-01-01,0,78741,30.22428901,-97.68704573,00:00:00
195558,IDENTITY THEFT,N,2020-01-01,0,78736,30.23947683,-97.89463328,00:00:00


In [ ]:
#drop old time column. 
crime_df.drop('occ_time',axis = 1,inplace=True)

C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
crime_df

,crime_type,family_violence,occ_date,zip_code,latitude,longitude,time_occ
1115,PUBLIC INTOXICATION,N,2021-12-31,78701,30.2672999,-97.73885705,23:50:00
1116,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,78701,30.26373894,-97.74365146,23:50:00
1117,THEFT,N,2021-12-31,78741,30.21526412,-97.70301942,23:50:00
1118,DOC DISCHARGE GUN - PUB PLACE,N,2021-12-31,78752,30.32804875,-97.69368286,23:47:00
1119,AGG ASLT STRANGLE/SUFFOCATE,Y,2021-12-31,78758,30.36926216,-97.69510501,23:40:00
...,...,...,...,...,...,...,...
195555,THEFT OF BICYCLE,N,2020-01-01,78703,30.27315833,-97.76014527,00:00:00
195556,HARASSMENT,N,2020-01-01,78741,30.2156655,-97.70354685,00:00:00
195557,THEFT FROM AUTO,N,2020-01-01,78741,30.22428901,-97.68704573,00:00:00
195558,IDENTITY THEFT,N,2020-01-01,78736,30.23947683,-97.89463328,00:00:00


In [ ]:
# Convert latitude and longitude as float
crime_df['latitude'] = crime_df['latitude'].astype('float')
crime_df['longitude'] = crime_df['longitude'].astype('float')
crime_df.dtypes


C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


crime_type                 object
family_violence            object
occ_date           datetime64[ns]
zip_code                   object
latitude                  float64
longitude                 float64
time_occ                   object
dtype: object

In [ ]:
# Converting zip_codes to integers
crime_df['zip_code'] = crime_df['zip_code'].astype('int64')
crime_df.dtypes


C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


crime_type                 object
family_violence            object
occ_date           datetime64[ns]
zip_code                    int64
latitude                  float64
longitude                 float64
time_occ                   object
dtype: object

In [ ]:
# Reorganize column info

column_order = ['crime_type', 'family_violence', 'occ_date', 'time_occ', 'zip_code','latitude','longitude']
crime_df = crime_df[column_order]
crime_df.groupby(['zip_code']).count()

,crime_type,family_violence,occ_date,time_occ,latitude,longitude
zip_code,,,,,,
78610,10,10,10,10,10,10
78613,866,866,866,866,866,866
78616,1,1,1,1,1,1
78617,1661,1661,1661,1661,1661,1661
78641,2,2,2,2,2,2
78652,65,65,65,65,65,65
78653,198,198,198,198,198,198
78660,1573,1573,1573,1573,1573,1573
78664,4,4,4,4,4,4


In [ ]:
#grouping crime type by zipcode

crime_by_zipcode_df = crime_df.groupby(['zip_code', 'crime_type']).count()
crime_by_zipcode_df

family_violence  occ_date  time_occ  \
zip_code crime_type                                                            
78610    ASSAULT W/INJURY-FAM/DATE VIOL                1         1         1   
         CRED CARD ABUSE - OTHER                       1         1         1   
         DWI                                           1         1         1   
         DWI 2ND                                       1         1         1   
         EVADING / VEHICLE PURSUIT                     1         1         1   
...                                                  ...       ...       ...   
78759    VIOL STATE LAW - OTHER                        4         4         4   
         VIOL TEMP EX PARTE  ORDER                     1         1         1   
         VOYEURISM-PEEPING TOM                         1         1         1   
         WARRANT ARREST NON TRAFFIC                   10        10        10   
         WEAPON VIOL - OTHER                           1         1         1   

                                         latitude  longitude  
zip_code crime_type                                           
78610    ASSAULT W/INJURY-FAM/DATE VIOL         1          1  
         CRED CARD ABUSE - OTHER                1          1  
         DWI                                    1          1  
         DWI 2ND                                1          1  
         EVADING / VEHICLE PURSUIT              1          1  
...                                           ...        ...  
78759    VIOL STATE LAW - OTHER                 4          4  
         VIOL TEMP EX PARTE  ORDER              1          1  
         VOYEURISM-PEEPING TOM                  1          1  
         WARRANT ARREST NON TRAFFIC            10         10  
         WEAPON VIOL - OTHER                    1          1  

[5872 rows x 5 columns]

In [ ]:
import pandas as pd
zip_code_avg_inc = pd.read_csv('Avg_INcome_by_Zip_Code_zip_atlas.csv')
zip_code_avg_inc

,#,Zip Code,Location,City,Population,Avg. Income/H/hold,National Rank
0,1,78730,"30.363632, -97.850355","Austin, Texas","4,885","$128,524.00",#58
1,2,78732,"30.382536, -97.894904","Austin, Texas","3,629","$103,951.00",#209
2,3,78739,"30.178839, -97.889064","Austin, Texas","8,643","$102,707.00",#227
3,4,78738,"30.327510, -97.989040","Austin, Texas","2,840","$102,295.00",#231
4,5,78733,"30.323857, -97.894860","Austin, Texas","8,716","$102,239.00",#232
5,6,78746,"30.296951, -97.811647","Austin, Texas","26,023","$100,571.00",#255
6,7,78726,"30.430488, -97.842530","Austin, Texas","6,480","$89,891.00",#462
7,8,78717,"30.488309, -97.764829","Austin, Texas","8,148","$87,290.00",#546
8,9,78737,"30.177041, -97.956661","Austin, Texas","7,100","$87,029.00",#555
9,10,78750,"30.413499, -97.802001","Austin, Texas","23,476","$78,428.00",#914


In [ ]:
result = pd.merge(crime_df, zip_code_avg_inc, how="inner", left_on=['zip_code'], right_on=['Zip Code'])
result.drop(['Zip Code', 'City', 'Location', '#'], axis = 1, inplace=True)
result = result[['crime_type', 'family_violence', 'occ_date', 'time_occ', 'zip_code', 'Population', 'Avg. Income/H/hold', 'National Rank', 'latitude' , 'longitude']]
result


,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude
0,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,"3,855","$35,757.00","#16,509",30.267300,-97.738857
1,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,"3,855","$35,757.00","#16,509",30.263739,-97.743651
2,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,"3,855","$35,757.00","#16,509",30.269900,-97.748559
3,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,"3,855","$35,757.00","#16,509",30.268218,-97.738693
4,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,"3,855","$35,757.00","#16,509",30.264693,-97.737904
...,...,...,...,...,...,...,...,...,...,...
186034,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,16:54:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978
186035,BOATING WHILE INTOXICATED,N,2020-04-10,20:50:00,78733,"8,716","$102,239.00",#232,30.353315,-97.858720
186036,BURGLARY OF VEHICLE,N,2020-04-10,13:40:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978
186037,CHILD CUSTODY INTERFERE,N,2020-04-03,18:00:00,78733,"8,716","$102,239.00",#232,30.334920,-97.870245


In [ ]:
# Read food_final_tx.csv
census_zip = pd.read_csv('food_final_tx.csv')
census_zip = census_zip.sort_values(['ZIP'])
census_zip.head()

,CensusTract,State,County,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,ZIP
10709,48453002307,Texas,Travis County,1,1,1,1,1,73301
10734,48453002411,Texas,Travis County,1,1,1,1,0,73301
9316,48421950200,Texas,Sherman County,0,0,0,0,0,73960
2756,48113013616,Texas,Dallas County,1,0,0,0,0,75001
2796,48113013725,Texas,Dallas County,1,0,1,0,1,75001


In [ ]:
#group census zones by zip codes and sum the food deficient zones
census_grouping = census_zip.groupby(['ZIP'],as_index=False).sum()
census_grouping.head()


,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle
0,73301,96906004718,2,2,2,2,1
1,73960,48421950200,0,0,0,0,0
2,75001,288678082405,6,0,1,0,1
3,75002,673190439628,14,1,1,1,1
4,75006,673582199329,14,0,6,0,1


In [ ]:
# Add total count of food deprived tracts. 
census_grouping['Low_access_tracts'] = census_grouping['LILATracts_1And10'] + census_grouping['LILATracts_halfAnd10'] + census_grouping['LILATracts_1And20'] + census_grouping['LILATracts_Vehicle']
census_grouping.head()

,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Low_access_tracts
0,73301,96906004718,2,2,2,2,1,7
1,73960,48421950200,0,0,0,0,0,0
2,75001,288678082405,6,0,1,0,1,2
3,75002,673190439628,14,1,1,1,1,4
4,75006,673582199329,14,0,6,0,1,7


In [ ]:
# Calculate the total tracts WITHOUT changing the order

total_tracts = census_zip['ZIP'].value_counts().sort_index()*4
total_tracts

73301     8
73960     4
75001    24
75002    56
75006    56
         ..
79955     4
79968     4
79995     4
79996     4
79997     4
Name: ZIP, Length: 2425, dtype: int64

In [ ]:
# get total count of tracts per zip code
total = pd.DataFrame(total_tracts, census_grouping['ZIP']).set_index(census_grouping.index)
total



,ZIP
0,8
1,4
2,24
3,56
4,56
...,...
2420,4
2421,4
2422,4
2423,4


In [ ]:
# total_tracts added to the dataframe.

census_grouping['total_tracts'] = (total)
census_grouping.head(10)


,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Low_access_tracts,total_tracts
0,73301,96906004718,2,2,2,2,1,7,8
1,73960,48421950200,0,0,0,0,0,0,4
2,75001,288678082405,6,0,1,0,1,2,24
3,75002,673190439628,14,1,1,1,1,4,56
4,75006,673582199329,14,0,6,0,1,7,56
5,75007,1058602462151,22,2,5,2,1,10,88
6,75009,240461141218,2,0,0,0,0,0,20
7,75010,481210216271,10,0,0,0,0,0,40
8,75011,96234035351,2,0,0,0,0,0,8
9,75013,288510189061,6,1,1,1,1,4,24


In [ ]:
census_grouping['Percentage_dep'] = (census_grouping['Low_access_tracts']/census_grouping['total_tracts']*100)


# Initiate list 
food_deprived_50 =[]
food_deprived_25 =[]
food_deprived_10 = []

# For loop to catogorize zip code food disparity.
for i in census_grouping['Percentage_dep']:
          if i >= 50:
                    i = 'Yes'
          else:
                    i='No'
          food_deprived_50.append(i)

for i in census_grouping['Percentage_dep']:
          if i >= 25:
                    i = 'Yes'
          else:
                    i='No'
          food_deprived_25.append(i)

for i in census_grouping['Percentage_dep']:
          if i >= 10:
                    i = 'Yes'
          else:
                    i='No'
          food_deprived_10.append(i)


census_grouping['food_deprived_50'] = food_deprived_50
census_grouping['food_deprived_25'] = food_deprived_25
census_grouping['food_deprived_10'] = food_deprived_10

census_grouping
                    

,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,73301,96906004718,2,2,2,2,1,7,8,87.500000,Yes,Yes,Yes
1,73960,48421950200,0,0,0,0,0,0,4,0.000000,No,No,No
2,75001,288678082405,6,0,1,0,1,2,24,8.333333,No,No,No
3,75002,673190439628,14,1,1,1,1,4,56,7.142857,No,No,No
4,75006,673582199329,14,0,6,0,1,7,56,12.500000,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,79955,48141001700,1,0,1,0,0,1,4,25.000000,No,Yes,Yes
2421,79968,48141001502,1,0,0,0,0,0,4,0.000000,No,No,No
2422,79995,48141003000,1,0,1,0,1,2,4,50.000000,Yes,Yes,Yes
2423,79996,48141010326,1,0,0,0,0,0,4,0.000000,No,No,No


In [ ]:
# Keep only the info we need
cleaned_census_data = census_grouping[['ZIP', 'Low_access_tracts', 'total_tracts', 'Percentage_dep','food_deprived_50','food_deprived_25','food_deprived_10']]
cleaned_census_data.head(20)

,ZIP,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,73301,7,8,87.500000,Yes,Yes,Yes
1,73960,0,4,0.000000,No,No,No
2,75001,2,24,8.333333,No,No,No
3,75002,4,56,7.142857,No,No,No
4,75006,7,56,12.500000,No,No,Yes
5,75007,10,88,11.363636,No,No,Yes
6,75009,0,20,0.000000,No,No,No
7,75010,0,40,0.000000,No,No,No
8,75011,0,8,0.000000,No,No,No
9,75013,4,24,16.666667,No,No,Yes


In [150]:
#Merge data frames together 

crime_df_cleaner = pd.merge(result, cleaned_census_data, how="inner", left_on=['zip_code'], right_on=['ZIP'])
crime_df_cleaner

,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude,ZIP,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,"3,855","$35,757.00","#16,509",30.267300,-97.738857,78701,7,32,21.875,No,No,Yes
1,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,"3,855","$35,757.00","#16,509",30.263739,-97.743651,78701,7,32,21.875,No,No,Yes
2,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,"3,855","$35,757.00","#16,509",30.269900,-97.748559,78701,7,32,21.875,No,No,Yes
3,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,"3,855","$35,757.00","#16,509",30.268218,-97.738693,78701,7,32,21.875,No,No,Yes
4,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,"3,855","$35,757.00","#16,509",30.264693,-97.737904,78701,7,32,21.875,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186034,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,16:54:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978,78733,0,16,0.000,No,No,No
186035,BOATING WHILE INTOXICATED,N,2020-04-10,20:50:00,78733,"8,716","$102,239.00",#232,30.353315,-97.858720,78733,0,16,0.000,No,No,No
186036,BURGLARY OF VEHICLE,N,2020-04-10,13:40:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978,78733,0,16,0.000,No,No,No
186037,CHILD CUSTODY INTERFERE,N,2020-04-03,18:00:00,78733,"8,716","$102,239.00",#232,30.334920,-97.870245,78733,0,16,0.000,No,No,No


In [151]:
#formatting new data frame. 
crime_df_cleaner.drop('ZIP',axis=1, inplace=True)
crime_df_cleaner.drop_duplicates(inplace=True)
crime_df_cleaner

,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,"3,855","$35,757.00","#16,509",30.267300,-97.738857,7,32,21.875,No,No,Yes
1,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,"3,855","$35,757.00","#16,509",30.263739,-97.743651,7,32,21.875,No,No,Yes
2,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,"3,855","$35,757.00","#16,509",30.269900,-97.748559,7,32,21.875,No,No,Yes
3,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,"3,855","$35,757.00","#16,509",30.268218,-97.738693,7,32,21.875,No,No,Yes
4,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,"3,855","$35,757.00","#16,509",30.264693,-97.737904,7,32,21.875,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186034,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,16:54:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978,0,16,0.000,No,No,No
186035,BOATING WHILE INTOXICATED,N,2020-04-10,20:50:00,78733,"8,716","$102,239.00",#232,30.353315,-97.858720,0,16,0.000,No,No,No
186036,BURGLARY OF VEHICLE,N,2020-04-10,13:40:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978,0,16,0.000,No,No,No
186037,CHILD CUSTODY INTERFERE,N,2020-04-03,18:00:00,78733,"8,716","$102,239.00",#232,30.334920,-97.870245,0,16,0.000,No,No,No


In [152]:
#Formatting different columns.

#Remove # from national rank
crime_df_cleaner['National Rank'] = crime_df_cleaner['National Rank'].replace({'\#':''},regex = True)
crime_df_cleaner['National Rank'] = crime_df_cleaner['National Rank'].replace({'\,':''},regex = True).astype('int')

# Round food deprived to the hundreths place
crime_df_cleaner['Percentage_dep']= crime_df_cleaner['Percentage_dep'].round(2)

# Remove , and $ from average household value
crime_df_cleaner['Avg. Income/H/hold'] = crime_df_cleaner['Avg. Income/H/hold'].replace({'\$':''}, regex = True)
crime_df_cleaner['Avg. Income/H/hold'] = crime_df_cleaner['Avg. Income/H/hold'].replace({'\,':''}, regex = True).astype('float')

#Remove , from population 
crime_df_cleaner['Population'] = crime_df_cleaner['Population'].replace({'\,':''}, regex = True).astype('int')

crime_df_cleaner['Avg. Income/H/hold'] = crime_df_cleaner['Avg. Income/H/hold'].replace({'\,':''}, regex = True).astype('float')
crime_df_cleaner


,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.267300,-97.738857,7,32,21.88,No,No,Yes
1,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.263739,-97.743651,7,32,21.88,No,No,Yes
2,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,3855,35757.0,16509,30.269900,-97.748559,7,32,21.88,No,No,Yes
3,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,3855,35757.0,16509,30.268218,-97.738693,7,32,21.88,No,No,Yes
4,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,3855,35757.0,16509,30.264693,-97.737904,7,32,21.88,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186034,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,16:54:00,78733,8716,102239.0,232,30.339078,-97.884978,0,16,0.00,No,No,No
186035,BOATING WHILE INTOXICATED,N,2020-04-10,20:50:00,78733,8716,102239.0,232,30.353315,-97.858720,0,16,0.00,No,No,No
186036,BURGLARY OF VEHICLE,N,2020-04-10,13:40:00,78733,8716,102239.0,232,30.339078,-97.884978,0,16,0.00,No,No,No
186037,CHILD CUSTODY INTERFERE,N,2020-04-03,18:00:00,78733,8716,102239.0,232,30.334920,-97.870245,0,16,0.00,No,No,No


In [153]:
# Statistical analysis of the crime_df_cleaner
Crime_stats= crime_df_cleaner.describe()
Crime_stats.drop(columns=['zip_code','latitude', 'longitude'], axis=1, inplace=True)
Crime_stats


,Population,Avg. Income/H/hold,National Rank,Low_access_tracts,total_tracts,Percentage_dep
count,185698.000000,185698.000000,185698.000000,185698.000000,185698.000000,185698.000000
mean,29430.242997,42449.209948,14092.666528,10.699399,47.696001,22.669005
std,14495.060877,16899.411718,9003.880446,6.722253,17.371541,16.447995
min,625.000000,14740.000000,58.000000,0.000000,4.000000,0.000000
25%,19585.000000,34242.000000,8326.000000,6.000000,32.000000,13.330000
50%,30110.000000,38206.000000,13714.000000,9.000000,52.000000,23.440000
75%,42820.000000,45090.000000,18353.000000,17.000000,56.000000,28.120000
max,53044.000000,128524.000000,31414.000000,22.000000,84.000000,95.000000


In [ ]:
crime_df_cleaner.to_csv('crime_data_cleaned_expanded.csv', index=False)
crime_by_zipcode_df.to_csv("crime_zipcode_data_expanded.csv")